#  Rosenbrock function optimization (with constraints).

The general equation is given by:

$f(x, y) = (1 - x)^2 + 100(y - x^2)^2$, with $-1.5 \le x \le +1.5$ and $-1.5 \le y \le +1.5$. The problem here is that we are trying to minimize this function subject to the following constraint: $x^2 + y^2 \le 2$. To do this we apply the [Penalty method](https://en.wikipedia.org/wiki/Penalty_method). Within this setting the global minimum is found at $f(1.0, 1.0) = 0.0$.

### First we import python libraries and set up the directory of our code.

In [1]:
import os, sys
import numpy as np
from math import fsum
import matplotlib.pyplot as plt

PROJECT_DIR = os.path.abspath('../code')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [2]:
# Import main classes.
from src.genome.gene import Gene
from src.genome.chromosome import Chromosome
from src.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from src.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from src.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from src.operators.mutation.random_mutator import RandomMutator

### Define the Rosebrock function, which plays also the role of the 'fitness' function.

In addition, we define the '_func' which takes the role of the 'random()' method of the Genes. Every time we
want to 'mutate' a gene this function will be called that returns 'valid', but random values for the gene.

In [3]:
# Rosenbrock function.
def fun_Rosenbrock(chromosome, f_min: bool = True):
    
    # Penalty coefficient.
    p_c = 0.5
    
    # Extract gene values as 'x' and 'y', for parsimony.
    x, y = chromosome[0].datum, chromosome[1].datum
    
    # Compute the function value.
    f_val = (1.0 - x)**2 + 100*(y - x**2)**2
    
    # Compute the constraint.
    f_const = p_c*max(0.0, x**2 + y**2 - 2.0)**2
    
    # NOTE: the constraint is added with the penalty coefficient.
    return -(f_val + f_const) if f_min else f_val
# _end_def_

# Random function ~U(-1.5, +1.5).
_func = lambda: np.random.uniform(-1.5, +1.5001)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has the same random() function (set by '_func'). But if the problem demands otherwise it is easy to set a 
different random() function for each gene.

In [4]:
# Define the number of chromosomes.
N = 100

# Initial population.
# It is important to note that the initial population is randomly generated with valid values.
population = [Chromosome([Gene(np.random.normal(), _func),
                          Gene(np.random.normal(), _func)], np.nan, True)
              for i in range(N)]

# Create the StandardGA object that will carry on the optimization.
# Note: in this example we have changed the mutation_probability to 0.15.
toy_GA = StandardGA(initial_pop=population, fit_func=fun_Rosenbrock, select_op=LinearRankSelector(),
                    mutate_op=RandomMutator(0.15), cross_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter,
such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [ ]:
toy_GA(epochs=10000, elitism=True, f_tol=1.0e-8, verbose=False)

Initial Avg. Fitness = -383.9081


In [ ]:
# Display the (final) optimal value.
print(f"Minimum Found: {fun_Rosenbrock(toy_GA.best_chromosome(), f_min=False):.5f}\n")

# Display each gene value separately.
for i, xi in enumerate(toy_GA.best_chromosome()._genome):
    print(f"x{i} = {xi.datum:.5f}")
# _end_for_

# True minimum: f(1.0, 1.0) = 0.0

In [ ]:
# If we want we can also print some operator statistics.
toy_GA.print_operator_stats()

In [ ]:
# Plot the average (population) fitness along with its spread (std)
# as function of the epochs.
plt.errorbar(x=range(len(toy_GA.stats['avg'])),
             y=toy_GA.stats['avg'], yerr=toy_GA.stats['std'])

plt.grid(True)
plt.title("Rosenbrock-GA")
plt.xlabel("Epoch")
plt.ylabel("$Mean~Fitness \pm Std$");

### End of file